In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import base64
import io
import torch
from io import StringIO
from io import BytesIO
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import ast
import os
import cv2
import albumentations as A
import numpy as np

In [4]:
%cd /content/drive/MyDrive/Colab_Skin_Disease/

/content/drive/MyDrive/Colab_Skin_Disease


In [5]:
def base64_to_image(base64_string):
    image_data = base64.b64decode(base64_string)
    image_buffer = BytesIO(image_data)
    image = Image.open(image_buffer)
    return image

In [6]:
def inference(model, img, conf, iou,img_size):
  model.conf = conf
  model.iou = iou
  # img = base64_to_image(img)
  results = model(img,size=img_size)
  results.print()
  results.show(labels=False)
  df = results.pandas().xyxy[0]
  return df

In [7]:
def multi_inference(model, imgs, conf, iou,img_size):
  model.conf = conf
  model.iou = iou
  results = model(imgs,size=img_size)
  results.print()
  results.show(labels=False)
  # results.save()
  df = results.pandas().xyxy[0]
  # return df

In [8]:
def df_to_dict(df):
  # Convert DataFrame to a list of dictionaries
  data_dict = df.to_dict(orient='records')
  return data_dict

In [9]:
def plot_annots(df,image_path):
  img = Image.open(image_path)
  fig, ax = plt.subplots(figsize=(5, 5))
  ax.imshow(img)
  ax.axis('off')

  for index,row in df.iterrows():
    xmin = float(row['xmin'])
    xmax = float(row['xmax'])
    ymin = float(row['ymin'])
    ymax = float(row['ymax'])

    print(xmax,xmin,ymax,ymin)
    rect = patches.Rectangle((xmin, ymin), (xmax-xmin),(ymax-ymin), linewidth=0.5, edgecolor='r')
    ax.add_artist(rect)

  plt.show()

In [10]:
def plot(image_path, annots, key):
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, 2, figsize=(8, 5))
    ax[0].imshow(img)
    ax[0].axis('off')
    for i in annots:
        xmin, ymin, xmax, ymax = i["xmin"],i["ymin"], i["xmax"], i["ymax"]

        rect = patches.Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), linewidth=0.5, edgecolor='r')
        ax[0].add_artist(rect)
    print(xmin, ymin, xmax, ymax)
    print(key)
    key = Image.open(key)
    ax[1].imshow(key)
    ax[1].axis('off')

    plt.show()

In [11]:
def face_crop(face_cascade,img_path):
    image = cv2.imread(img_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(15, 15))

    if len(faces) > 0:
        # Select the first detected face
        (x, y, w, h) = faces[0]

        # Draw rectangle around the detected face
        # cv2.rectangle(image, (x-50, y-50), (x+w+50, y+h+50), (0, 0, 255), 2)

        # Crop the image within the bounding box
        cropped_image = image[max(0, y-50):y+h+50, max(0, x-50):x+w+50]

        # Convert the cropped image from BGR to RGB for displaying in matplotlib
        cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

        # Display the cropped image
        # plt.imshow(cropped_image_rgb)
        # plt.axis('off')
        # plt.show()
        return cropped_image_rgb
    else:
        print("No face detected in the image.")
        return None

In [ ]:
!pip install ultralytics

In [23]:
weights = '/content/drive/MyDrive/Colab_Skin_Disease/yolov5/runs/train/exp94/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path = weights)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-31 Python-3.10.6 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 212 layers, 20865057 parameters, 0 gradients
Adding AutoShape... 


In [24]:
file_paths = []
for dirpath,_,filenames in os.walk('/content/drive/MyDrive/Colab_Skin_Disease/TEST Images/'):
  for f in filenames:
      file_paths.append(os.path.abspath(os.path.join(dirpath, f)))

In [25]:
#converting img to base64 just for testing
# img_path = '/content/drive/MyDrive/Colab_Skin_Disease/via-2.0.12/kkkk2.jpg'
img_path = '/content/drive/MyDrive/Colab_Skin_Disease/TEST Images/IMG_1566.jpg'

# with open(img_path, "rb") as img_file:
#     img_64 = base64.b64encode(img_file.read())

In [27]:
# image = Image.open(img_path)
image =  plt.imread(img_path)

## Load the pre-trained face detection cascade
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# image = face_crop(face_cascade,img_path)

transform = A.Compose([
 A.CLAHE(clip_limit=(2,2), tile_grid_size=(8,8),always_apply=True),
#  A.RandomRotate90(always_apply=True)
])
# image = np.array(image)
image = transform(image=image)['image']

df = inference(model,image,conf=0.1,iou=0.2,img_size=640)
display(df)
plot_annots(df=df,image_path=img_path)

# annots = df_to_dict(df)
# key = '/content/drive/MyDrive/Colab_Skin_Disease/All_diseases_key.drawio.png'
# print(annots)
# plot(img_path,annots,key)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
for path in file_paths:
  # image =  plt.imread(path)

  # transform = A.Compose([
  #   A.CLAHE(clip_limit=(2,2), tile_grid_size=(8,8),always_apply=True),
  #   # A.RandomRotate90(always_apply=True)
  # ])
  # trans_image = transform(image=image)['image']

  # df = inference(model,trans_image,conf=0.15,iou=0.2,img_size=1024)
  # # break
  multi_inference(model,path,conf=0.1,iou=0.25,img_size=640)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for path in file_paths:
   image = face_crop(face_cascade,path)
   if image is not None:
      multi_inference(model,image,conf=0.15,iou=0.2,img_size=10)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the pre-trained face detection cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the image
img_path = '/content/drive/MyDrive/Colab_Skin_Disease/TEST Images/IMG_1566.jpg'
image = cv2.imread(img_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Perform face detection
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))

if len(faces) > 0:
    # Select the first detected face
    (x, y, w, h) = faces[0]

    # Draw rectangle around the detected face
    cv2.rectangle(image, (x-50, y-50), (x+w+50, y+h+50), (0, 0, 255), 2)

    # Crop the image within the bounding box
    # cropped_image = image[max(0, y-50):y+h+50, max(0, x-50):x+w+50]

    # Convert the cropped image from BGR to RGB for displaying in matplotlib
    # cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the cropped image
    plt.imshow(image)
    plt.axis('off')
    plt.show()
else:
    print("No face detected in the image.")


In [ ]:
from pixellib.tune_bg import alter_bg

change_bg = alter_bg(model_type = "pb")
change_bg.load_pascalvoc_model("../input/dataset/xception_pascalvoc.pb")
output = change_bg.change_bg_img(f_image_path = "../input/images/c5afc3a4_2285_11ec_b946_80e650049a6e.png",b_image_path = "../input/images/bg.jpeg", detect = "car")
cv2.imwrite("img.jpg", output)
im_rgb = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
image = Image.fromarray(im_rgb)
image

In [ ]:
import pixellib
from pixellib.semantic import semantic_segmentation

segment_image = semantic_segmentation()
segment_image.load_pascalvoc_model("deeplabv3_xception_tf_dim_ordering_tf_kernels.h5")
segment_image.segmentAsPascalvoc("path_to_image", output_image_name = "path_to_output_image")